In [1]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from fastai.vision.gan import *
from PIL import Image

import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import pdb

# Discriminator

In [ ]:
class MultiUNITDiscriminator(nn.Module):
    def __init__(self):
        super(MultiUNITDiscriminator, self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),
            conv_and_res(64, 128),
            conv_and_res(128, 256),
            conv_and_res(256, 512),
            nn.Conv2d(512, 1, 3, stride=1),
            Flatten()
        )
    
    def forward(self, not_switched, switched, down=2):

        not_switched = self.convs(not_switched)
        switched = self.convs(switched)

        return (not_switched,switched)

In [ ]:
class critic_loss(nn.Module):
    #a is 0 and b is 1 for predictions
    def forward(self, output, garbage):
        pred_winter = output[0]
        pred_summer = output[1]
        targWin = pred_winter.new_zeros(*pred_winter.size())
        targSum = pred_summer.new_ones(*pred_summer.size())
        result_winter = F.mse_loss(pred_winter, targWin)
        result_summer = F.mse_loss(pred_summer, targSum)
        return result_winter + result_summer

In [ ]:
critic_learner = Learner(data, MultiUNITDiscriminator(), loss_func=critic_loss(), wd=1e-3)